# 📦 1. Import necessary libraries

In [14]:
import requests
import json
from datetime import timedelta
import pandas as pd
from ApiKey import api_key

# ------------------- CONFIG -------------------

# 🔐 2. Configure API key, headers, players name and url

In [140]:
# We create the headers that PUBG requires to authenticate and specify the format
HEADERS = {
    "Authorization": f"Bearer {api_key}",
    "Accept": "application/vnd.api+json"
}

# Name of player to search
player_name  = 'arbigegegegegege'
#player_name  = 'KaDiz-'
#player_name  = 'XioYuanLinPay'

# URL base to the PUBG API
base_url = "https://api.pubg.com/shards/steam"

# ------------------- EXTRACT -------------------

# 🎮 3. Find player ID

In [144]:
def searchPlayer (player_name):
    # Endpoint to find player for name
    url = f"{base_url}/players?filter[playerNames]={player_name}"
    
    # Send the GET request to the API with authorization headers
    response = requests.get(url, headers=HEADERS)

    # Raise an exception if the request failed (e.g., 404 or 500)
    response.raise_for_status()    

    #Save the information and transform with Json library and return the player id
    data = response.json()
    return data["data"][0]["id"]
    
searchPlayer(player_name)

'account.57c8e062f89d4fee80e393332f12cdc9'

# 🎯 4. Obtain matches from the player

In [ ]:
def find_match_players(player_id):
    # Endpoint to obtain data of the player from id
    url = f"{base_url}/players/{player_id}"

    # Send the GET request to the API with authorization headers
    response = requests.get(url, headers=HEADERS)
    
    # Raise an exception if the request failed (e.g., 404 or 500)
    response.raise_for_status()  

    # Returns all of matches from the player
    return response.json()["data"]["relationships"]["matches"]["data"]

find_match_players(searchPlayer(player_name))


# 📊 5. Obtain match details

In [226]:
def extract_match_details(match_id):
    # Construct the API endpoint URL using the match ID
    url = f"{base_url}/matches/{match_id}"
    
    # Send the GET request to the API with authorization headers
    response = requests.get(url, headers=HEADERS)
    
    # Raise an exception if the request failed (e.g., 404 or 500)
    response.raise_for_status()
    
    # Return the JSON response containing match details
    return response.json()

extract_match_details(find_match_players(searchPlayer(player_name))[0]['id'])


{'data': {'type': 'match',
  'id': 'ea1fd759-1346-4b28-92d3-a391f395d1f5',
  'attributes': {'duration': 1800,
   'gameMode': 'squad-fpp',
   'mapName': 'Desert_Main',
   'createdAt': '2025-04-27T23:57:30Z',
   'titleId': 'bluehole-pubg',
   'shardId': 'steam',
   'tags': None,
   'isCustomMatch': False,
   'matchType': 'competitive',
   'seasonState': 'progress',
   'stats': None},
  'relationships': {'rosters': {'data': [{'type': 'roster',
      'id': '1a611de5-8a44-4b3f-b2a0-3b5a139b1285'},
     {'type': 'roster', 'id': '6f9f51c2-f02e-4f88-92d9-94820ec42989'},
     {'type': 'roster', 'id': 'eb8cae06-94de-4b17-90a2-19cf26e6e5d1'},
     {'type': 'roster', 'id': '31d1b458-5ad8-4d2a-880d-4d3ea7251b2b'},
     {'type': 'roster', 'id': 'e192f35b-16b5-49c5-8072-8b8ed48b4085'},
     {'type': 'roster', 'id': 'cbbd9627-1f01-4aec-8cb6-d24dbfc9f611'},
     {'type': 'roster', 'id': '8a223fae-6692-4a8b-8159-7565b1f9ff65'},
     {'type': 'roster', 'id': '47a0169d-6142-4c06-8909-1d9410f4561d'},
     

# ------------------- TRANSFORM -------------------

In [240]:
def transform_match(match_data):
    participants = [x for x in match_data['included'] if x['type'] == 'participant']
    rows = []
    for p in participants:
        stats = p['attributes']['stats']
        rows.append({
            "name": stats['name'],
            "playerId": stats["playerId"],
            "kills": stats['kills'],
            "longestKill": stats["longestKill"],
            "killPlace": stats["killPlace"],
            "assists": stats["assists"],
            "damageDealt": stats['damageDealt'],
            "winPlace": stats['winPlace'],
            "timeSurvived": str(pd.to_timedelta(stats['timeSurvived'], unit='s')).split()[-1],
            "heals": stats["heals"],
            "boosts": stats["boosts"],
            "deathType": stats["deathType"],
            "walkDistance": stats["walkDistance"],
            "rideDistance": stats["rideDistance"],
            "swimDistance": stats["swimDistance"],
            "vehicleDestroys": stats["vehicleDestroys"]
        })
    return pd.DataFrame(rows)

transform_match(extract_match_details(find_match_players(searchPlayer(player_name))[0]['id']))

,name,playerId,kills,longestKill,killPlace,assists,damageDealt,winPlace,timeSurvived,heals,boosts,deathType,walkDistance,rideDistance,swimDistance,vehicleDestroys
0,x_ALBY_x,account.bb83f8919a0a44f7a0a3d96880695e61,0,0.000000,34,0,95.138855,8,00:15:19,1,3,byplayer,995.729740,3226.1360,0,0
1,MOEIN-TTV,account.a1411b1140fa414eb6a845ddb96b8642,0,0.000000,47,0,61.091595,14,00:04:55,1,2,byplayer,267.523470,0.0000,0,0
2,arbigegegegegege,account.57c8e062f89d4fee80e393332f12cdc9,2,58.968110,12,2,765.893700,3,00:29:45,15,10,byplayer,1098.934300,393.2712,0,0
3,suntzu99,account.7c0c59f4d9a64b49a013fc07fb7511a8,0,0.000000,63,0,98.641980,18,00:01:23,0,0,byplayer,18.605658,0.0000,0,0
4,CrzDimsz,account.720d03dd0b524b3baf9c591b0c55f552,1,94.470590,21,0,565.955300,1,00:26:59,8,13,byplayer,407.677280,0.0000,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,C4-Brown,account.198bb0f996754dc5b959b87bcd16ea82,2,27.667805,11,0,282.249150,2,00:25:08,2,7,byplayer,1248.034700,0.0000,0,0
58,Ex1st7,account.b74d89cac0614c6f9956c9a7cfd2f235,3,69.886640,5,2,479.983150,7,00:07:32,2,2,byplayer,488.065730,0.0000,0,0
59,THE-DEAD-MAN_,account.cb207828b6fd4b23ba115616c3f98302,0,0.000000,40,2,0.000000,11,00:07:34,1,2,byplayer,1026.486900,0.0000,0,0
60,BR1GHTS1D3,account.c35603f56ab645b7bdeafb3749baaae6,0,0.000000,44,0,49.876183,12,00:06:18,1,1,suicide,527.291900,0.0000,0,0


# 📊 5.2 Mostrar la media de los ultimos 10 matchs

In [44]:
# Variables para acumular los resultados
total_kills = 0
total_damage = 0
total_time_survived = 0
total_ranking = 0
total_wins = 0
total_kill_place = 0
total_kill_streaks = 0
num_matches = 0  # Para contar el número de partidas procesadas

# Recorremos cada match_id para obtener información de la partida
for match_id  in match_ids:
    # Endpoint para obtener datos del match
    url = f"https://api.pubg.com/shards/steam/matches/{match_id}"
    
    # Hacemos la petición para obtener los datos de la partida
    response = requests.get(url, headers=headers)
    
    # Convertimos la respuesta a JSON
    match_data = response.json()
    
    # Accedemos a la información del jugador dentro de esa partida
    participants = match_data["included"]

    # Buscamos al jugador en esa partida (comparando player_id)
    for player in participants:
        if player["type"] == "participant" and "stats" in player["attributes"]:
            stats = player["attributes"]["stats"]
           
            # Verifica si el jugador es el que estás buscando
            if stats.get("playerId") == player_id:
                kills = stats.get("kills", 0)
                killPlace = stats.get("killPlace", 0)
                killStreaks = stats.get("killStreaks", 0)
                damage = stats.get("damageDealt", 0)
                duracion = timedelta(seconds=stats.get("timeSurvived", 0))
                ranking = stats.get("winPlace", 0)
                win = ranking == 1
                
                # Acumular los valores
                total_kills += kills
                total_damage += damage
                total_time_survived += stats.get("timeSurvived", 0)
                total_ranking += ranking
                total_wins += win
                total_kill_place += killPlace
                total_kill_streaks += killStreaks
                num_matches += 1

# Calcular las medias
if num_matches > 0:
    avg_kills = total_kills / num_matches
    avg_damage = total_damage / num_matches
    avg_time_survived = timedelta(seconds=(total_time_survived / num_matches))
    avg_ranking = total_ranking / num_matches
    avg_wins = total_wins / num_matches
    avg_kill_place = total_kill_place / num_matches
    avg_kill_streaks = total_kill_streaks / num_matches

    # Mostrar las medias
    print("Average Stats over the last 10 matches:")
    print(f"Average Kills: {avg_kills:.2f}")
    print(f"Average Damage: {avg_damage:.2f}")
    print(f"Average Time Survived: {avg_time_survived}")
    print(f"Average Ranking: {avg_ranking:.2f}")
    print(f"Average Wins: {avg_wins:.2f}")
    print(f"Average Kill Place: {avg_kill_place:.2f}")
    print(f"Average Kill Streaks: {avg_kill_streaks:.2f}")
else:
    print("No matches found.")

NameError: name 'match_ids' is not defined

# 📊 5.5 Mostrar detalles del primer match obteniendo todos los jugadores y equipos

In [42]:
# Recorremos cada match_id para obtener información de la partida
for match_id  in last_Match_id:
    # Endpoint para obtener datos del match
    url = f"https://api.pubg.com/shards/steam/matches/{match_id}"
    
    # Hacemos la petición para obtener los datos de la partida
    response = requests.get(url, headers=headers)
    
    # Convertimos la respuesta a JSON
    match_data = response.json()
    
    # Buscamos al jugador en esa partida (comparando player_id)
    roster_match = match_data["included"]
    all_participants = []  # ← Aquí guardaremos todos los participantes

    # Primero, recogemos todos los participantes
    for data in roster_match:
        if data["type"] == "participant":
            all_participants.append(data)

    # Usamos un set para llevar control de los jugadores ya procesados y evitar duplicados
    seen_players = set()

    # Suponiendo que tienes una lista de rosters
    for data in roster_match:  
        if data["type"] == "roster":
            team_id = data["id"]
            team_rank = data["attributes"]["stats"].get("rank", "Desconocido")
            team_Id = data["attributes"]["stats"].get("teamId", "Desconocido")
            team_win = data["attributes"]["won"] == "true"
            participants = data["relationships"]["participants"]["data"]

            print(f"Match ID: {match_id}")
            print(f"team id: {team_id}")
            print(f"team nº: {team_Id}")
            print(f"team rank: {team_rank}")
            print(f"Win?: {'Yes' if team_win else 'No'} \n")   

            #Buscar jugadores en el roster y sus estadísticas
            for player_info in participants:
                player_id = player_info["id"]  # ID del jugador en el roster
                
                # Evitamos imprimir jugadores repetidos
                if player_id in seen_players:
                    continue
                seen_players.add(player_id)  # Lo marcamos como "visto"

                # Buscar el participante con ese ID
                player = next((p for p in all_participants if p["id"] == player_id), None)

                if not player or "attributes" not in player or "stats" not in player["attributes"]:
                    continue

                stats = player["attributes"]["stats"]
                kills = stats.get("kills", 0)
                name_player = stats.get("name", "Desconocido")
                damage = stats.get("damageDealt", 0)
                ranking = stats.get("winPlace", 0)
                duracion = timedelta(seconds=stats.get("timeSurvived", 0))
                win = ranking == 1

                # Mostrar detalles del jugador
                print(f"Player ID: {player_id}")
                print(f"Player Name: {name_player}")
                print(f"Kills: {kills}")
                print(f"Damage: {damage}")
                print(f"Time survived: {duracion}")
                print(f"Ranking: {ranking} \n")

NameError: name 'last_Match_id' is not defined

# Adaptando el codigo a Pandas

In [160]:
print(json.dumps(dataOriginal, indent=4))

NameError: name 'dataOriginal' is not defined

In [ ]:
# Suponiendo que tu variable es 'data' 
players = dataOriginal["data"]

# Normalizamos todo lo posible, separando las claves anidadas con "_"
df = pd.json_normalize(players, sep="_")

# Mostramos el DataFrame completo
df


In [ ]:
# Normalizar los matches dentro de relationships.matches.data
df_matches = pd.json_normalize(
    players,
    record_path=["relationships", "matches", "data"],
    sep="_"
)

# Mostramos el DataFrame completo
df_matches

In [38]:
# URL para obtener las seasons
urlSeasonActual = "https://api.pubg.com/shards/steam/seasons"


# Hacemos la petición
responseActualSeason = requests.get(urlSeasonActual, headers=headers)

# Verificamos y extraemos el seasonId actual
if responseActualSeason.status_code == 200:
    seasons = responseActualSeason.json()
    
    # Buscamos la temporada actual
    season_id = None
    for s in seasons["data"]:
        if s["attributes"].get("isCurrentSeason"):
            season_id = s["id"]
            break

    if season_id is None:
        print("No se encontró la temporada actual.")
    else:
        # Armamos el endpoint con el season actual
        endpointSeason = f"https://api.pubg.com/shards/steam/players/{player_id}/seasons/{season_id}/ranked"

        # Hacemos la petición
        response = requests.get(endpointSeason, headers=headers)

        if response.status_code == 200:
            dataSeason = response.json()
            print("Todo Correcto, arcivhos guardado en data")
        else:
            print("Error en ranked:", response.status_code)
else:
    print("Error al obtener seasons:", responseActualSeason.status_code)


Todo Correcto, arcivhos guardado en data


In [40]:
print(json.dumps(dataSeason, indent=4))

{
    "data": {
        "type": "rankedplayerstats",
        "attributes": {
            "rankedGameModeStats": {
                "squad-fpp": {
                    "currentTier": {
                        "tier": "Diamond",
                        "subTier": "3"
                    },
                    "currentRankPoint": 3247,
                    "bestTier": {
                        "tier": "Diamond",
                        "subTier": "3"
                    },
                    "bestRankPoint": 3247,
                    "roundsPlayed": 58,
                    "avgRank": 6.793103,
                    "avgSurvivalTime": 0,
                    "top10Ratio": 0.7586207,
                    "winRatio": 0.13793103,
                    "assists": 48,
                    "wins": 8,
                    "kda": 4,
                    "kdr": 0,
                    "kills": 156,
                    "deaths": 51,
                    "roundMostKills": 0,
                    "longestKill": 0,
